In [1]:
import numpy as np
import pandas as pd
import re
import csv
import tldextract
import torch
from src.NeuralNetwork import NeuralNetwork
from src.urlfunctions import *
import urllib
from urllib.parse import urlparse

email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_domain(url):
    o = urllib.parse.urlsplit(url)
    return o.hostname, tldextract.extract(url).domain, o.path

def words_raw_extraction(domain, subdomain, path):
    w_domain = re.split("\-|\.|\/|\?|\=|\@|\&|\%|\:|\_", domain.lower())
    w_subdomain = re.split("\-|\.|\/|\?|\=|\@|\&|\%|\:|\_", subdomain.lower())   
    w_path = re.split("\-|\.|\/|\?|\=|\@|\&|\%|\:|\_", path.lower())
    raw_words = w_domain + w_path + w_subdomain
    w_host = w_domain + w_subdomain
    raw_words = list(filter(None,raw_words))
    return raw_words, list(filter(None,w_host)), list(filter(None,w_path))

allbrand_txt = open("data/allbrands.txt", "r")

def __txt_to_list(txt_object):
    list = []
    for line in txt_object:
        list.append(line.strip())
    txt_object.close()
    return list

allbrand = __txt_to_list(allbrand_txt)

def domain_in_brand(domain):   
    if domain in allbrand:
        return 1
    else:
        return 0
    
def brand_in_path(domain,path):
    for b in allbrand:
        if '.'+b+'.' in path and b not in domain:
           return 1
    return 0

In [40]:
url = "http://www.crestonwood.com/router.php"

hostname, domain, path = get_domain(url)
tldinfo = tldextract.extract(url)
domain = tldinfo.domain+'.'+tldinfo.suffix
subdomain = tldinfo.subdomain
tld = tldinfo.suffix

tmp = url[url.find(tldinfo.suffix):len(url)]
pth = tmp.partition("/")
path = pth[1] + pth[2]

words_raw, words_raw_host, words_raw_path= words_raw_extraction(tldinfo.domain, subdomain, pth[2])

parsed = urlparse(url)
scheme = parsed.scheme

In [ ]:
s = """url.count('/')
url.count('*')
url.count(":")
url.count(',')
url.count(';')
url.count('$')
url.count(' ')
url.count('www')
url.count('com')
count_double_slash(url)
count_http_token(url)
https_token(scheme)
punycode(url)
port(url)
tld_in_path(tld,path)
tld_in_subdomain(tld,subdomain)
abnormal_subdomain(url)
count_subdomain(url)
prefix_suffix(url)
shortening_service(url)
char_repeat(words_raw)
phish_hints(url)
domain_in_brand(domain)
brand_in_path(subdomain)
brand_in_path(path)
suspecious_tld(tld)
whois_registered_domain(domain)
domain_registration_length(domain)
domain_age(domain)
web_traffic(url)
dns_record(domain)
google_index(url)
0"""

l=s.split("\n")
#print(l)
for i in range(len(l)):
    #print(f"output[{12+i}] = {l[i]}")
    pass

In [47]:
output = np.ndarray(45, dtype=int)
output[0] = len(url)
output[1] = len(hostname)
output[2] = url.count(".")
output[3] = url.count("-")
output[4] = url.count("@")
output[5] = url.count("?")
output[6] = url.count("&")
output[7] = url.count('|')
output[8] = url.count('=')
output[9] = url.count("_")
output[10] = url.count("~")
output[11] = url.count("%")
output[12] = url.count('/')
output[13] = url.count('*')
output[14] = url.count(":")
output[15] = url.count(',')
output[16] = url.count(';')
output[17] = url.count('$')
output[18] = url.count(' ')
output[19] = url.count('www')
output[20] = url.count('com')
output[21] = count_double_slash(url)
output[22] = count_http_token(url)
output[23] = https_token(scheme)
output[24] = punycode(url)
output[25] = port(url)
output[26] = tld_in_path(tld,path)
output[27] = tld_in_subdomain(tld,subdomain)
output[28] = abnormal_subdomain(url)
output[29] = count_subdomain(url)
output[30] = prefix_suffix(url)
output[31] = shortening_service(url)
output[32] = char_repeat(words_raw)
output[33] = phish_hints(url)
output[34] = domain_in_brand(domain)
output[35] = brand_in_path(tldinfo.domain, subdomain)
output[36] = brand_in_path(tldinfo.domain, path)
output[37] = suspecious_tld(tld)
output[38] = whois_registered_domain(domain)
output[39] = domain_registration_length(domain)
output[40] = -2 #domain_age(domain)
output[41] = web_traffic(url)
output[42] = dns_record(domain)
output[43] = google_index(url)
output[44] = 4 # page_rank when we get openpagerank api key


m_input = output

In [48]:
input_tensor = torch.tensor(m_input, dtype=torch.float32).to(device)
#input_tensor = input_tensor.view(1, -1)

input_size = 45
output_size = 1

hidden_size = 40
num_hidden_layers = 10
model_path = "trained_weights/neural/scn_0.002_11_10_40.pth"

model = NeuralNetwork(input_size, hidden_size, output_size, num_hidden_layers).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()
with torch.no_grad():
    output = model(input_tensor)

print(torch.sigmoid(output))
predicted_class = torch.sigmoid(output) >= 0.5

print("Model prediction for the input:", predicted_class.item())


tensor([0.4167], device='cuda:0')
Model prediction for the input: False


In [45]:
data_array = np.load("website_data_more.npy")
print(data_array.shape)
features = data_array[:,:-1]
print(features.shape)
print(m_input.shape)
print(features[0].shape)
print(m_input)
print(features[0])

num = 0

for i in range(45):
    print(f"{m_input[i]} : {features[0][i]}")
    
while True:
    break
    input_tensor = torch.tensor(features[num], dtype=torch.float32).to(device)
    #input_tensor = input_tensor.view(1, -1)

    with torch.no_grad():
        output = model(input_tensor)

    print(torch.sigmoid(output))
    predicted_class = torch.sigmoid(output) >= 0.5

    print("Model prediction for the input:", predicted_class.item())
    num+=1
    input()

(11161, 46)
(11161, 45)
(45,)
(45,)
[37 19  3  0  0  0  0  0  0  0  0  0  3  0  1  0  0  0  0  1  1  0  1  1
  0  0  0  0  0  3  0  0  4  0  0  0  0  0  1 -1 -2  0  0  1  0]
[37 19  3  0  0  0  0  0  0  0  0  0  3  0  1  0  0  0  0  1  0  0  0  1
  0  0  0  0  0  3  0  0  4  0  0  0  0  0  0 45 -1  0  1  1  4]
37 : 37
19 : 19
3 : 3
0 : 0
0 : 0
0 : 0
0 : 0
0 : 0
0 : 0
0 : 0
0 : 0
0 : 0
3 : 3
0 : 0
1 : 1
0 : 0
0 : 0
0 : 0
0 : 0
1 : 1
1 : 0
0 : 0
1 : 0
1 : 1
0 : 0
0 : 0
0 : 0
0 : 0
0 : 0
3 : 3
0 : 0
0 : 0
4 : 4
0 : 0
0 : 0
0 : 0
0 : 0
0 : 0
1 : 0
-1 : 45
-2 : -1
0 : 0
0 : 1
1 : 1
0 : 4
